In [2]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense

In [26]:
data = {
    "year": [2012, 2014, 2016, 2018, 2020, 2022],
    "Voter_reg": [1313850, 1300455, 1379230, 1428856, 1564297, 1529401],
    "turnout": [888033, 529927, 998605, 813353, 1166203, 716322],
    "%turnout": [67.6, 40.7, 72.4, 56.9, 74.6, 46.8],
    "unemp_rate": [6.3, 6.1, 5.1, 3.4, 8.1, 2.6],
    "Democrats_house": [85020, 107390, 129760, 130743, 166758, 101404],
    "Republicans_house": [138488, 148202.5, 157917, 115588, 176141, 136038],
    "per_capita_income": [39954, 41971, 44087, 53008, 56363, 70497]
}
df = pd.DataFrame(data)

In [27]:
# Preprocessing
# Normalize data using MinMaxScaler
scaler = MinMaxScaler()
scaled_data = scaler.fit_transform(df.drop(columns=["year"]))

# Convert data into sequences for LSTM
def create_sequences(data, target_columns, seq_length=2):
    X, y = [], []
    for i in range(len(data) - seq_length):
        X.append(data[i:i+seq_length])
        y.append(data[i+seq_length, target_columns])
    return np.array(X), np.array(y)

# Target columns (Democrats_house: 4, Republicans_house: 5)
X, y = create_sequences(scaled_data, target_columns=[4, 5])

# LSTM Model
model = Sequential([
    LSTM(64, activation='relu', input_shape=(X.shape[1], X.shape[2]), return_sequences=False),
    Dense(32, activation='relu'),
    Dense(2)  # Predicting two outputs: Democrats_house and Republicans_house
])
model.compile(optimizer='adam', loss='mse', metrics=['mae'])

# Train the model on the entire dataset
history = model.fit(X, y, epochs=100, batch_size=1, verbose=0)

# Predict the next values using the last sequence
predicted_scaled = model.predict(scaled_data[-2:].reshape(1, 2, -1))  # Predict the next time step
predicted = scaler.inverse_transform(np.hstack([scaled_data[-1, :-2], predicted_scaled[0]]).reshape(1, -1))

# Display predictions
print("Predicted values for Democrats_house and Republicans_house:"+str(predicted[0, 4])+' '+ str(predicted[0, 5]))


/usr/local/lib/python3.10/dist-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step   
Predicted values for Democrats_house and Republicans_house:101404.0 197237.93930590153
